In [1]:
with open('lolita.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#$%&'()*,-./0123456789:;<=>?ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_abcdefghijklmnopqrstuvwxyz{|}~£©«®°»—‘’“”„•€™■►
110


In [5]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

'hi there'

In [7]:
import torch
data = torch.tensor(encode(text),dtype=torch.long)
print(data.shape)
print(data)

torch.Size([662979])
tensor([ 0, 50, 73,  ...,  0,  0,  0])


In [9]:
n = int(.9*len(data))
train_data = data[:n]
test_data = data[n:]

In [10]:
block_size = 8
train_data[:block_size+1]

tensor([ 0, 50, 73, 71, 78,  1, 82, 77,  1])

In [14]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else test_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb,yb = get_batch('train')
print(xb)
print(yb)
    
    

tensor([[67, 63, 80, 81,  1, 66, 83, 80],
        [74, 66,  1, 75, 63, 73, 67,  1],
        [77, 83, 76, 69,  1, 70, 83, 81],
        [67, 74, 78, 74, 67, 81, 81,  1]])
tensor([[63, 80, 81,  1, 66, 83, 80, 71],
        [66,  1, 75, 63, 73, 67,  1, 77],
        [83, 76, 69,  1, 70, 83, 81, 64],
        [74, 78, 74, 67, 81, 81,  1, 63]])
